In [1]:
import scipy.io as sci2
from scipy import signal
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as mtp
import random as rng
import functools
from sklearn import svm
from sklearn.lda import LDA
from sklearn.decomposition import PCA
from sklearn import preprocessing
from scipy.stats import pearsonr
from sklearn import cross_validation

from sklearn.feature_selection import RFE, SelectKBest
from sklearn.linear_model import LogisticRegression

import os
%matplotlib inline  

E:\Miniconda\lib\site-packages\sklearn\lda.py:6: DeprecationWarning: lda.LDA has been moved to discriminant_analysis.LinearDiscriminantAnalysis in 0.17 and will be removed in 0.19
  "in 0.17 and will be removed in 0.19", DeprecationWarning)
E:\Miniconda\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# transforma tempo em índice de array 
#    seconds => tempo em segundos
#    fs => frequencia de amostragem
def indexFromSeconds(seconds, fs):
    return np.multiply(seconds,fs)

# calcula o intervalo médio entre spikes
#    spikes => array crescente com os tempos (em qualquer unidade) de ocorrência de spikes em um determinado neurônio
#    retorna 0 se o número de spikes for <= 1
#    retorna o intervalo médio, caso contrário
def spikesLatency(spikes):
    if len(spikes) <= 1:
        return 0.0
    s = 0
    latencies = []
    for i in range(0, len(spikes) - 1):
        latencies.append(spikes[i + 1] - spikes[i])
        s += spikes[i + 1] - spikes[i]
    return s/len(spikes)

# extrai a somatório do espectro de potência nas faixas: [4-8] Hz, [8-12] Hz, [12-40] Hz, [40-100] Hz
# flp1 => np array com 4 colunas cada uma contendo um sinal lfp         Obs.: Convenção linha x coluna: (linha, coluna)
# fs => frequência de amostragem
# OBS.: essa função pode ser generalizada para um número arbitrário de lfps e para bandas de frequência arbitrárias
def lfpBandsExtractor(flp1, fs):
    #bands = [[0,4], [4,8], [8,12], [12,40], [40,100]]
    bands = [[4,8], [8,12], [12,40], [40,100]]        #excluindo banda delta
    bandsV = []
    for j in range(0, 4):
        f = np.fft.fft(flp1[:, j])
        f = np.abs(f)
        f = f * f

        time_step = 1/fs
        freqs = np.fft.fftfreq(len(flp1), time_step)
        idx = np.argsort(freqs)
        x = freqs[idx]
        y = f[idx]

        i = 0
        #mtp.plot(x, y)
        #mtp.xlabel('Frequência')
        #mtp.ylabel('Potência')
        #np.savetxt(fname='LFP_power_example', delimiter='\n', fmt='%.12f', newline='\n', X=y)
        #mtp.clf()
        #mtp.plot(flp1[:,0])
        #mtp.xlabel('Tempo')
        #mtp.ylabel('Potencial')
        #np.savetxt(fname='LFP_example', delimiter='\n', fmt='%.12f', newline='\n', X=flp1[:,0])
        for b in bands:
            bandsum = 0
            while(x[i] < b[0]):
                i += 1
            while(x[i] < b[1]):
                bandsum += y[i]
                i += 1
            bandsV.append(bandsum)
    
    return bandsV 

In [3]:
# nova implementação do classificador que agora não faz distinção entre dados pré e dados pós trigtimes (contínua)
#    mat_full => dicionario contendo todos os spikes de todos os neuronios gravados de um animal
#    mat => sinais LFP's
#    trigtimes => tempos nos quais o animal tocou o nose poke da câmara central (s)
#    labels => Do mesmo tamanho de 'trigtimes', indica para que lado o animal foi no trial (0 ou 1)
#    fs => frequência de amostragem (Hz) (dos sinais LFP's)
#    interesting_time_start => indica quanto tempo antes de cada trigtime a janela deslizante irá começar (s)
#    interesting_time_end => indica quanto tempo depois de cada trigtime a janela deslizante irá parar (s)
#    window_size => tamanho da janela deslizante (s)
#    overlap => tamanho do avanço da janela deslizante (s)
#    subject_name => nome do animal (só para gerar arquivos customizados)
def classifyOverlappingContinuous(mat_full, mat, trigtimes, incorrectTrigtimes, labels, incorrectLabels, fs, interesting_time_start, 
                                  interesting_time_end, window_size, overlap, subject_name):
       
    #np.random.shuffle(labels)  #para testar viés do classificador
    
    #selecionando os indices dos 30 melhores atributos
    top_attrs = rankings(mat_full, mat, trigtimes, labels, 1000, 0, 1, 1, 1, subject_name)[0:30] #melhores 30 atributos entre 0s e 1s pós trigtimes
    
    # criando diretórios para guardar resultados intermediários e finais 
    if not os.path.exists('ranking'):
        os.makedirs('ranking')
    if not os.path.exists('datasets'):
        os.makedirs('datasets')
    if not os.path.exists('ranking/' + subject_name):
        os.makedirs('ranking/' + subject_name)
        os.makedirs('datasets/' + subject_name)
    
    #com deslize
    if (overlap > 0):
        numberOfLoops = int((interesting_time_end - interesting_time_start - window_size)/overlap)   #a divisão tem que dar inteira, atenção
    else:
        numberOfLoops = 0    #sem deslize (interessante para rodar a função uma única vez em um intervalo de tempo ex.: extrair os mellhores atributos)
    print(numberOfLoops)
    
    # monta uma lista com os nomes de cada atributo
    data_header = []
    for l in spikes_labels:
        data_header.append(l + '_num_spikes')
        data_header.append(l + '_lat_spikes')
    ##data_header.append('delta_power1')
    #data_header.append('theta_power1')
    #data_header.append('alpha_power1')
    #data_header.append('beta_power1')
    #data_header.append('gamma_power1')
    ##data_header.append('delta_power2')
    #data_header.append('theta_power2')
    #data_header.append('alpha_power2')
    #data_header.append('beta_power2')
    #data_header.append('gamma_power2')
    ##data_header.append('delta_power3')
    #data_header.append('theta_power3')
    #data_header.append('alpha_power3')
    #data_header.append('beta_power3')
    #data_header.append('gamma_power3')
    ##data_header.append('delta_power4')
    #data_header.append('theta_power4')
    #data_header.append('alpha_power4')
    #data_header.append('beta_power4')
    #data_header.append('gamma_power4')
    data_header.append('decision')
    
    #guardará a taxa de acerto para cada trial
    total_scores = []
    #guardará a taxa de acerto para cada trial com os atributos selecionados
    better_total_scores = []
    #guardará a taxa de acerto do modelo treinado com trials corretos prevendo os trials incorretos
    correct_incorrect_scores = []
    
    fileNumber = 1
    for loop in range(0, numberOfLoops + 1):   # +1 accounts for the first window, so if the number of loops == 0, it will run once
        #trigtimes
        eventIndexes = trigtimes[:,0]
        #dataset for each trigtime (evento)
        dataframe = []
        
        # pra cada trial ... extraia o dataset
        for j in range(0, len(eventIndexes)):
            print("\r",j,end='')
            #guarda cada feature na ordem de data_header
            event_features = []
            #para cada neurônio, extrair os atributos: "numero de spikes", "tempo médio entre spikes"
            for i in range(0, len(spikes_labels)):
                spikeTimes = np.array(mat_full[spikes_labels[i]]) #in seconds
                #fft lfp attributes
                spikesCount = 0
                SpikesList = []
                
                SpikesList = spikeTimes[(spikeTimes >= eventIndexes[j] + interesting_time_start + (loop * overlap)) &
                                          (spikeTimes < eventIndexes[j] + interesting_time_start + window_size + (loop * overlap))]
                
                event_features.append(len(SpikesList))
                event_features.append(spikesLatency(SpikesList))                
            #extrai a potência de cada faixa de frequência de cada sinal LFP e concatena na lista de atributos  
            #event_features = event_features + lfpBandsExtractor(
            #    mat[eventIndexes[j]*fs + interesting_time_start*fs + loop*overlap*fs:
            #        eventIndexes[j]*fs + interesting_time_start*fs +  window_size*fs + loop*overlap*fs], fs)
            #concatena os labels no fim da lista de atributos
            event_features.append(labels[j,0])
            #adiciona a linha de atributos desse trial para o dataset
            dataframe.append(event_features)
        
        #### INCORRECT DATAFRAME ####
        
        dataframeIncorrect = []
        
        # pra cada trial ... extraia o dataset
        for j in range(0, len(incorrectTrigtimes)):
            #guarda cada feature na ordem de data_header
            eventIncorrect_features = []
            #para cada neurônio, extrair os atributos: "numero de spikes", "tempo médio entre spikes"
            for i in range(0, len(spikes_labels)):
                spikeTimes = np.array(mat_full[spikes_labels[i]]) #in seconds
                #fft lfp attributes
                spikesCount = 0
                SpikesList = []
                
                SpikesList = spikeTimes[(spikeTimes >= incorrectTrigtimes[j] + interesting_time_start + (loop * overlap)) &
                                          (spikeTimes < incorrectTrigtimes[j] + interesting_time_start + window_size + (loop * overlap))]
                
                eventIncorrect_features.append(len(SpikesList))
                eventIncorrect_features.append(spikesLatency(SpikesList))                
            #extrai a potência de cada faixa de frequência de cada sinal LFP e concatena na lista de atributos  
            #event_features = event_features + lfpBandsExtractor(
            #    mat[eventIndexes[j]*fs + interesting_time_start*fs + loop*overlap*fs:
            #        eventIndexes[j]*fs + interesting_time_start*fs +  window_size*fs + loop*overlap*fs], fs)
            #concatena os labels no fim da lista de atributos
            eventIncorrect_features.append(incorrectLabels[j])
            #adiciona a linha de atributos desse trial para o dataset
            dataframeIncorrect.append(eventIncorrect_features)
        
        ####  END INCORRECT SECTION  ####
        
        print("Correto", np.shape(dataframe))
        print("Incorreto:", np.shape(dataframeIncorrect))
        #salva o dataset na pasta datasets
        np.savetxt('datasets/' + subject_name + '/' + "{0:0=5d}".format(fileNumber) + '_dataset_' + str( interesting_time_start + (loop * overlap)) 
                   + '@' + str(interesting_time_start + window_size + (loop * overlap)) + '.csv', delimiter=',', fmt='%.8f',
                    newline='\n', header=",".join(data_header), X=dataframe)
        #salva o dataset incorreto na pasta datasetsIncorrects
        np.savetxt('datasetsIncorrects/' + subject_name + '/' + "{0:0=5d}".format(fileNumber) + '_dataset_' + str( interesting_time_start + (loop * overlap)) 
                   + '@' + str(interesting_time_start + window_size + (loop * overlap)) + '.csv', delimiter=',', fmt='%.8f',
                    newline='\n', header=",".join(data_header), X=dataframeIncorrect)
        #normalizing
        
        #transforma dataframe para numpy array
        features_set = np.asarray(dataframe)        
        features_set_incorrect = np.asarray(dataframeIncorrect)
        
        #começa processo de classificação com todos atributos
        print("    Rodando database normal...")
        #guarda todos os scores no processo de cross-validation
        all_scores = []
        #cross-validation usando k = raiz_quadrada da quantidade de trials
        k_fold = cross_validation.KFold(len(features_set), int(np.sqrt(np.shape(features_set)[0])), random_state=151515)
        for k, (train, test) in enumerate(k_fold):
            #cria classificador SVM default
            clf = svm.SVC()
            #normalizando dados
            scaler = preprocessing.StandardScaler().fit(features_set[train,0:np.shape(features_set)[1] - 1])
            #treinando svm
            clf.fit(scaler.transform(features_set[train,0:np.shape(features_set)[1] - 1]), features_set[train, np.shape(features_set)[1] - 1])
            #classificando...
            predictions = clf.predict(scaler.transform(features_set[test, 0:np.shape(features_set)[1] - 1]))
            #acertos (array de 0's e 1's, 0-erro e 1-acerto)
            scored = predictions == features_set[test, np.shape(features_set)[1] - 1]
            
            #calcula acerto entre 0 e 1 (foi trocado pelo F1-score)
            #all_scores.append(sum(scored)/len(scored))
            
            #calcula F-1 score (https://en.wikipedia.org/wiki/Precision_and_recall - tabela da direita)
            labels_before_prediction = features_set[test, np.shape(features_set)[1] - 1]
            negatives = labels_before_prediction[:] == 0
            positives = labels_before_prediction[:] == 1
            TP = sum(predictions[positives])
            FP = sum(predictions[negatives] == 1)
            FN = sum(predictions[positives] == 0)
            f1_score = (2*TP)/(2*TP + FP + FN)
            all_scores.append(f1_score)
        #averages all_scores, resultando na taxa de acerto média para a janela de tempo em questão 
        print("Taxa de acerto média ", int(np.sqrt(np.shape(features_set)[0])), "fold:", sum(all_scores)/len(all_scores))
        total_scores.append(sum(all_scores)/len(all_scores))
        
        #criando um classificador com os 30 melhores atributos
        print("    Rodando database com os melhores atributos...")
        
        #opcional, faz o ranqueamento usando regressão logistica, de todos os atributos para esse trial e salva em um arquivo na pasta ranking
        dataset_std = preprocessing.StandardScaler().fit_transform(features_set[:,0:np.shape(features_set)[1] - 1])
        model = LogisticRegression()
        rfe = RFE(model, 1)
        fit = rfe.fit(dataset_std, features_set[:, np.shape(features_set)[1] - 1])
        data_header = np.array(data_header)
        rankeds = np.argsort(fit.ranking_)
        
        np.savetxt('ranking/' + subject_name + '/' + "{0:0=5d}".format(fileNumber) + '_ranking_features_' + str(interesting_time_start + (loop * overlap)) 
                   + '@' + str(interesting_time_start + window_size + (loop * overlap)) + '.csv',
                   delimiter=',', fmt='%s', newline='\n', X=data_header[rankeds])
        #fim opcional (rankings faz a mesma coisa, só não salva no arquivo)
        
        #novos datasets (corretos e incorretos) utilizando os 30 melhores atributos
        filtered_dataset = dataset_std[:, top_attrs]
        dataset_std_incorrect = preprocessing.StandardScaler().fit_transform(features_set_incorrect[:,0:np.shape(features_set_incorrect)[1] - 1])
        filtered_dataset_incorrect = dataset_std_incorrect[:, top_attrs]
        
        #### modelo dos trials corretos tentará predizer os trials incorretos ####
        
        clf = svm.SVC()
        scaler = preprocessing.StandardScaler().fit(filtered_dataset[:,0:(filtered_dataset.shape[1] - 1)])
        #treinando o SVM com todos os trials corretos
        clf.fit(scaler.transform(filtered_dataset[:, 0:(filtered_dataset.shape[1] - 1)]), features_set[:,features_set.shape[1] - 1])
        predictions = clf.predict(scaler.transform(filtered_dataset_incorrect[:, 0:(filtered_dataset_incorrect.shape[1] - 1)]))
        
        scored = predictions == features_set_incorrect[:, features_set_incorrect.shape[1] - 1]
        labels_before_prediction = features_set_incorrect[:, np.shape(features_set_incorrect)[1] - 1]
        negatives = labels_before_prediction[:] == 0
        positives = labels_before_prediction[:] == 1
        TP = sum(predictions[positives])
        FP = sum(predictions[negatives] == 1)
        FN = sum(predictions[positives] == 0)
        f1_score = (2*TP)/(2*TP + FP + FN)
        
        correct_incorrect_scores.append(f1_score)
        #### ####
        
        all_scores = []
        k_fold = cross_validation.KFold(len(filtered_dataset), int(np.sqrt(np.shape(features_set)[0])), random_state=151515)
        for k, (train, test) in enumerate(k_fold):
            clf = svm.SVC()
            #print("treino:", len(train), "teste:", len(test))
            scaler = preprocessing.StandardScaler().fit(filtered_dataset[train,0:(filtered_dataset.shape[1] - 1)])
            clf.fit(scaler.transform(filtered_dataset[train, 0:(filtered_dataset.shape[1] - 1)]), features_set[train,features_set.shape[1] - 1])
            predictions = clf.predict(scaler.transform(filtered_dataset[test, 0:(filtered_dataset.shape[1] - 1)]))
            scored = predictions == features_set[test, features_set.shape[1] - 1]
            
            #print("Taxa de acerto cross: " + str(sum(scored)/len(scored)))
            #all_scores.append(sum(scored)/len(scored))
            
            #calcula F-1 score (https://en.wikipedia.org/wiki/Precision_and_recall - tabela da direita)
            labels_before_prediction = features_set[test, np.shape(features_set)[1] - 1]
            negatives = labels_before_prediction[:] == 0
            positives = labels_before_prediction[:] == 1
            TP = sum(predictions[positives])
            FP = sum(predictions[negatives] == 1)
            FN = sum(predictions[positives] == 0)
            f1_score = (2*TP)/(2*TP + FP + FN)
            all_scores.append(f1_score)
        
        print("Taxa de acerto média atributos selecionados", int(np.sqrt(np.shape(features_set)[0])), "fold:", sum(all_scores)/len(all_scores))
        better_total_scores.append(sum(all_scores)/len(all_scores))
        fileNumber += 1
    # retorna a curva da taxa de acerto dos classificadores sem seleção de atributos e com, respectivamente
    return total_scores, better_total_scores, correct_incorrect_scores

# segue a mesma documentação da função acima e faz a mesma coisa, só que no final, ranqueia os melhores atributos no processo de classificação
def rankings(mat_full, mat, trigtimes, labels, fs, interesting_time_start, 
                                  interesting_time_end, window_size, overlap, subject_name):
    
    #np.random.shuffle(labels)
    
    if not os.path.exists('ranking'):
        os.makedirs('ranking')
    if not os.path.exists('datasets'):
        os.makedirs('datasets')
    if not os.path.exists('datasetsIncorrects'):
        os.makedirs('datasetsIncorrects')
    if not os.path.exists('ranking/' + subject_name):
        os.makedirs('ranking/' + subject_name)
        os.makedirs('datasets/' + subject_name)
        os.makedirs('datasetsIncorrects/' + subject_name)
        
    #if (overlap > (interesting_time_end - interesting_time_start) - window_size):
    #    raise Error('Erro')
    if (overlap > 0):
        numberOfLoops = int((interesting_time_end - interesting_time_start - window_size)/overlap)   #a divisão tem que dar inteira, atenção
    else:
        numberOfLoops = 0
    print('Número de Loops no rankings', numberOfLoops)
    
    data_header = []
    for l in spikes_labels:
        data_header.append(l + '_num_spikes')
        data_header.append(l + '_lat_spikes')
    ##data_header.append('delta_power1')
    #data_header.append('theta_power1')
    #data_header.append('alpha_power1')
    #data_header.append('beta_power1')
    #data_header.append('gamma_power1')
    ##data_header.append('delta_power2')
    #data_header.append('theta_power2')
    #data_header.append('alpha_power2')
    #data_header.append('beta_power2')
    #data_header.append('gamma_power2')
    ##data_header.append('delta_power3')
    #data_header.append('theta_power3')
    #data_header.append('alpha_power3')
    #data_header.append('beta_power3')
    #data_header.append('gamma_power3')
    ##data_header.append('delta_power4')
    #data_header.append('theta_power4')
    #data_header.append('alpha_power4')
    #data_header.append('beta_power4')
    #data_header.append('gamma_power4')
    data_header.append('decision')
    
    total_scores = []
    better_total_scores = []
    for loop in range(0, numberOfLoops + 1):   # +1 accounts for the first window
        eventIndexes = trigtimes[:,0]
        results = ['D', 'E']
        dataframe = []

        searchIndexes = [0] * len(spikes_labels)

        for j in range(0, len(eventIndexes)):
            print("\r",j,end='')
            event_features = []
            for i in range(0, len(spikes_labels)):
                spikeTimes = np.array(mat_full[spikes_labels[i]]) #in seconds
                #fft lfp attributes
                spikesCount = 0
                posSpikesCount = 0
                preSpikesList = []
                
                #pre
                preSpikesList = spikeTimes[(spikeTimes >= eventIndexes[j] + interesting_time_start + (loop * overlap)) &
                                          (spikeTimes < eventIndexes[j] + interesting_time_start + window_size + (loop * overlap))]
                
                event_features.append(len(preSpikesList))
                event_features.append(spikesLatency(preSpikesList))                
            #pre    
            #event_features = event_features + lfpBandsExtractor(
            #    mat[eventIndexes[j]*fs + interesting_time_start*fs + loop*overlap*fs:
            #        eventIndexes[j]*fs + interesting_time_start*fs +  window_size*fs + loop*overlap*fs], fs)
            
            event_features.append(labels[j,0])            
            dataframe.append(event_features)        
        #normalizing
        
        features_set = np.asarray(dataframe)        
        
        #print(len(features_set))       
        
        #print("    Rodando database normal...")
        all_scores = []
        k_fold = cross_validation.KFold(len(features_set), int(np.sqrt(np.shape(features_set)[0])), random_state=151515)
        for k, (train, test) in enumerate(k_fold):
            clf = svm.SVC()
            #print("treino:", len(train), "teste:", len(test))
            scaler = preprocessing.StandardScaler().fit(features_set[train,0:np.shape(features_set)[1] - 1])
            clf.fit(scaler.transform(features_set[train,0:np.shape(features_set)[1] - 1]), features_set[train, np.shape(features_set)[1] - 1])
            predictions = clf.predict(scaler.transform(features_set[test, 0:np.shape(features_set)[1] - 1]))
            scored = predictions == features_set[test, np.shape(features_set)[1] - 1]
            #print("Taxa de acerto cross: " + str(sum(scored)/len(scored)))
            all_scores.append(sum(scored)/len(scored))
            #print(predictions)
        #scores = cross_validation.cross_val_score(
        #clf, X_scaled, features_set[:,164], cv=5)
        #print("Taxa de acerto média ", int(np.sqrt(np.shape(features_set)[0])), "fold:", sum(all_scores)/len(all_scores))
        total_scores.append(sum(all_scores)/len(all_scores))
        
        dataset_std = preprocessing.StandardScaler().fit_transform(features_set[:,0:np.shape(features_set)[1] - 1])
        model = LogisticRegression()
        rfe = RFE(model, 1)
        fit = rfe.fit(dataset_std, features_set[:, np.shape(features_set)[1] - 1])
        data_header = np.array(data_header)
        rankeds = np.argsort(fit.ranking_)
        return rankeds
        
        

In [4]:
#variáveis importantes para o funcionamento de todas as funções

#lista dos nomes de todos os arquivos ".mat" de spikes 
spikes_files = sorted(os.listdir("spikes_right"))
#lista dos nomes de todos os arquivos ".mat" de lfp's 
lfps_files = sorted(os.listdir("lfps"))
#lista dos nomes de todos os arquivos ".mat" de labels 
labels_files = sorted(os.listdir("labels"))
#lista dos nomes de todos os arquivos ".mat" de eventos discrimados incorretamente pelos animais
incorrect_files = sorted(os.listdir("incorrect/incorrect"))

In [5]:
a = sci2.loadmat('incorrect/incorrect/' + incorrect_files[0])['incorrectTrials']
len(a[:,1])

113

In [10]:


#para cada animal...
for f_index in range(8, 9):
#for f_index in range(0, len(spikes_files)):
    #carrega o dicionário de neurônios
    print(spikes_files[f_index])
    mat_full = sci2.loadmat('spikes_right/' + spikes_files[f_index])
    #carrega só os sinais LFP's do .mat de LFP
    mat = sci2.loadmat('lfps/' + lfps_files[f_index])['LFP']
    #carrega os labels
    labels = sci2.loadmat('labels/' + labels_files[f_index])['labels']
    #carrega os labels incorretos que representam o lado (errado) que o animal escolheu
    incorrectLabels = sci2.loadmat('incorrect/incorrect/' + incorrect_files[0])['incorrectTrials'][:,1]
    #carrega os trigtimes
    trigtimes = sci2.loadmat('labels/' + labels_files[f_index])['trigtimes']
    #carrega os trigtimes onde o animal errou
    incorrectTrigtimes = sci2.loadmat('incorrect/incorrect/' + incorrect_files[0])['incorrectTrials'][:,0]
    #filtra só as chaves importantes no dicionário do .mat de spikes
    spikes_labels = list(key for key in mat_full.keys() if key.startswith('SI')
                         or key.startswith('PFC') or key.startswith('VI') or key.startswith('PPC'))
    
    #pega as curvas da taxa de acerto para o animal spikes_files[f_index], de -4 a 4 segundos com uma janela de 0.5s deslizando em 0.025s
    learning_curve, better_learning_curve, correct_incorrect_scores = classifyOverlappingContinuous(mat_full, mat, trigtimes, incorrectTrigtimes, labels, incorrectLabels, 1000, -4, 4, 0.5, 0.025, spikes_files[f_index])
    
    #dataset, labels = classifyOverlappingContinuous(mat_full, mat, trigtimes, labels, 1000, -0.5, 4, 0.5, 0.025, spikes_files[f_index])
    #learning_curve, better_learning_curve = classifyOverlappingContinuous(mat_full, mat, trigtimes, labels, 1000, 0, 1, 1, 1, spikes_files[f_index])
    
    #guarda curvas em arquivos
    if not os.path.exists('learning-curves'):
        os.makedirs('learning-curves')
    if not os.path.exists('learning-curves/' + spikes_files[f_index].split('.')[0]):
        os.makedirs('learning-curves/' + spikes_files[f_index].split('.')[0])
    if not os.path.exists('correct-incorrect-curves'):
        os.makedirs('correct-incorrect-curves')
    if not os.path.exists('correct-incorrect-curves/' + spikes_files[f_index].split('.')[0]):
        os.makedirs('correct-incorrect-curves/' + spikes_files[f_index].split('.')[0])
    np.savetxt('learning-curves/' + spikes_files[f_index].split('.')[0] + '/' + 'posrates_4segs_0.5window_size_0.025overlapping.csv' , delimiter=',', fmt='%.14f', newline='\n', X=learning_curve)
    np.savetxt('learning-curves/' + spikes_files[f_index].split('.')[0] + '/' + 'better_posrates_4segs_0.5window_size_0.025overlapping.csv' , delimiter=',', fmt='%.14f', newline='\n', X=better_learning_curve)
    np.savetxt('correct-incorrect-curves/' + spikes_files[f_index].split('.')[0] + '/' + 'correct_incorrect_curve_4segs_0.5window_size_0.025overlapping.csv' , delimiter=',', fmt='%.14f', newline='\n', X=correct_incorrect_scores)

rat4-preto-WD-20150316-60min-239trials-92PC-SPK_data_right.mat
Número de Loops no rankings 0
 195300
 195131Correto (196, 165)
Incorreto: (113, 165)
    Rodando database normal...
Taxa de acerto média  14 fold: 0.585559130412
    Rodando database com os melhores atributos...
Taxa de acerto média atributos selecionados 14 fold: 0.471071423908
 195Correto (196, 165)
Incorreto: (113, 165)
    Rodando database normal...
Taxa de acerto média  14 fold: 0.529389098297
    Rodando database com os melhores atributos...
Taxa de acerto média atributos selecionados 14 fold: 0.437088110302
 195Correto (196, 165)
Incorreto: (113, 165)
    Rodando database normal...
Taxa de acerto média  14 fold: 0.545080153273
    Rodando database com os melhores atributos...
Taxa de acerto média atributos selecionados 14 fold: 0.476867670145
 195Correto (196, 165)
Incorreto: (113, 165)
    Rodando database normal...
Taxa de acerto média  14 fold: 0.549946274056
    Rodando database com os melhores atributos...
Taxa

In [ ]:
import random
def gen_hex_colour_code():
   return '#'+ ''.join([random.choice('0123456789abcdef') for x in range(6)])
gen_hex_colour_code()

In [ ]:
#proccess results

results_folder = "results_pos"
results_folder_pre = "results_pre"


results_files = os.listdir(results_folder)
results_files_pre = os.listdir(results_folder_pre)

rats = set()
for f in results_files:
    fn = f.split('WD')[0]
    fn = fn[0: len(fn) - 1]
    rats.add(fn)

better_curves = []
worse_curves = []
    
for rat in rats:
    print(rat)
    rat_files = list(k for k in results_files if k.startswith(rat))
    graphical_files = list(k for k in rat_files if 'overlapping' in k)
    sequencial_files = list(k for k in rat_files if 'overlapping' not in k)
    better_file = list(k for k in graphical_files if 'better' in k)
    worse_file = list(k for k in graphical_files if 'better' not in k)
    features_evolution = len(sequencial_files)*[None]
    
    rat_files_pre = list(k for k in results_files_pre if k.startswith(rat))
    graphical_files_pre = list(k for k in rat_files_pre if 'overlapping' in k)
    sequencial_files_pre = list(k for k in rat_files_pre if 'overlapping' not in k)
    better_file_pre = list(k for k in graphical_files_pre if 'better' in k)
    worse_file_pre = list(k for k in graphical_files_pre if 'better' not in k)
    features_evolution_pre = len(sequencial_files_pre)*[None]
    
    for sf in sequencial_files:
        ind = sf.split('_')
        ind = ind[len(ind) - 1]
        ind = int(ind.split('.')[0])
        features_evolution[ind] = np.loadtxt(results_folder+'/'+sf, dtype=bytes, delimiter="\n").astype(str)
    for sf in sequencial_files_pre:
        ind = sf.split('_')
        ind = ind[len(ind) - 1]
        ind = int(ind.split('.')[0])
        features_evolution_pre[ind] = np.loadtxt(results_folder_pre+'/'+sf, dtype=bytes, delimiter="\n").astype(str)
        
    better = np.loadtxt(results_folder+'/'+better_file[0])
    worse = np.loadtxt(results_folder+'/'+worse_file[0])
    
    better_pre = np.loadtxt(results_folder_pre+'/'+better_file_pre[0])
    worse_pre = np.loadtxt(results_folder_pre+'/'+worse_file_pre[0])
    
    better_curves.append(np.hstack((better_pre, better)))
    worse_curves.append(np.hstack((worse_pre, worse)))
    
    mtp.rcParams['xtick.labelsize'] = 14 
    mtp.rcParams['ytick.labelsize'] = 14 
    mtp.figure(figsize=(20,14))
    xaxis = np.linspace(0, 4, 141)
    mtp.title(rat + ' learning rates', fontsize=20)
    mtp.xlabel('Tempo (s)', fontsize=20)
    mtp.ylabel('Taxa de acerto [0,1]', fontsize=20)
    axes = mtp.gca()
    axes.set_ylim([0.4,1])
    axes.set_yticks(np.arange(0.4, 1.05, 0.05))
    axes.set_xticks(np.arange(-4, 4.25, 0.5))
    #mtp.yticks = np.arange(0.4, 1.05, 0.05)
    
    mtp.axvline(x=0, linewidth=0.5, linestyle='dashed', color='gray')
    
    for coord in np.arange(0.4, 1.05, 0.05):
        mtp.axhline(y=coord, linewidth=0.5, linestyle='dashed', color='blue')
    
    mtp.plot(xaxis, worse, 'r-',xaxis, better, 'g-', xaxis[::-1]*-1, worse_pre, 'r-', xaxis[::-1]*-1, better_pre, 'g-')
    mtp.savefig(rat + '_full_learning_rates.png', dpi=200)
    print('plotou')
    features_rankings = {}
    features_rankings_pre = {}
    for i in features_evolution[0]:
        features_rankings[i] = []
        features_rankings_pre[i] = []
    mtp.close()
    #rank of each feature per time
    for fe in range(0, len(features_evolution)):
        for rank in range(0, len(features_evolution[fe])):
            features_rankings[features_evolution[fe][rank]].append(rank)
    #mtp.figure(figsize=(30,20))
    #for feature in (list(features_rankings.keys())[0:4]):
    #    mtp.plot(features_rankings[feature])
    #mtp.savefig('kappapride', dpi=200)
    
    #mtp.clf()  
    
    for fe in range(0, len(features_evolution_pre)):
        for rank in range(0, len(features_evolution_pre[fe])):
            features_rankings_pre[features_evolution_pre[fe][rank]].append(rank)
    #mtp.figure(figsize=(30,20))
    #for feature in (list(features_rankings_pre.keys())[0:4]):
    #    mtp.plot(features_rankings_pre[feature])
    #mtp.savefig('kappapride2', dpi=200)
    
    #mtp.clf()
    
    #appearance of each feature at top30 over time
    all_labels = list(features_rankings.keys())
    print("Rato: ", rat, "Eletrodos: ", (len(all_labels) - 20)/2)
    continue
    #print(all_labels)
    mtp.figure(figsize=(20,14))
    top = 30
    axes = mtp.gca()
    axes.set_xticks(np.arange(-4, 4.5, 0.5))
    #for i in range(0, len(features_rankings[all_labels[0]])):
    #    y_values = []
    #    for j in range(0, len(all_labels)):
    #        if features_rankings[all_labels[j]][i] < top:
    #            y_values.append(j)
    #        else:
    #            y_values.append(0)
    #    mtp.plot([i] * len(y_values), y_values, 'b*')    
    
    regions_evolution = {}
    
    regions_evolution['PFC'] = [0] * len(features_rankings[all_labels[0]])
    regions_evolution['PPC'] = [0] * len(features_rankings[all_labels[0]])
    regions_evolution['SI'] = [0] * len(features_rankings[all_labels[0]])
    regions_evolution['VI'] = [0] * len(features_rankings[all_labels[0]])
    #regions_evolution['LFP_related'] = [0] * len(features_rankings[all_labels[0]])
    #measuring region importance
    for i in range(0, len(features_rankings[all_labels[0]])):
        for lab in all_labels:
            if(features_rankings[lab][i] < top):
                if(lab.startswith('PFC')):
                    regions_evolution['PFC'][i] += 1
                elif(lab.startswith('PPC')):
                    regions_evolution['PPC'][i] += 1
                elif(lab.startswith('SI')):
                    regions_evolution['SI'][i] += 1
                elif(lab.startswith('VI')):
                    regions_evolution['VI'][i] += 1
                elif(lab.endswith('1')):
                    regions_evolution_pre['PFC'][i] += 1
                elif(lab.endswith('2')):
                    regions_evolution_pre['PPC'][i] += 1
                elif(lab.endswith('3')):
                    regions_evolution_pre['SI'][i] += 1
                elif(lab.endswith('4')):
                    regions_evolution_pre['VI'][i] += 1
    
    #for region in regions_evolution.keys():
    #    mtp.plot(regions_evolution[region], label=region )
    #mtp.legend()
    #mtp.savefig(rat + '_regions_importance_evolution_pos.png', dpi=200)
    
    #mtp.clf()
    regions_evolution_pre = {}
    
    regions_evolution_pre['PFC'] = [0] * len(features_rankings_pre[all_labels[0]])
    regions_evolution_pre['PPC'] = [0] * len(features_rankings_pre[all_labels[0]])
    regions_evolution_pre['SI'] = [0] * len(features_rankings_pre[all_labels[0]])
    regions_evolution_pre['VI'] = [0] * len(features_rankings_pre[all_labels[0]])
    #regions_evolution_pre['LFP_related'] = [0] * len(features_rankings_pre[all_labels[0]])
    #measuring region importance
    for i in range(0, len(features_rankings_pre[all_labels[0]])):
        for lab in all_labels:
            if(features_rankings_pre[lab][i] < top):
                if(lab.startswith('PFC')):
                    regions_evolution_pre['PFC'][i] += 1
                elif(lab.startswith('PPC')):
                    regions_evolution_pre['PPC'][i] += 1
                elif(lab.startswith('SI')):
                    regions_evolution_pre['SI'][i] += 1
                elif(lab.startswith('VI')):
                    regions_evolution_pre['VI'][i] += 1
                elif(lab.endswith('1')):
                    regions_evolution_pre['PFC'][i] += 1
                elif(lab.endswith('2')):
                    regions_evolution_pre['PPC'][i] += 1
                elif(lab.endswith('3')):
                    regions_evolution_pre['SI'][i] += 1
                elif(lab.endswith('4')):
                    regions_evolution_pre['VI'][i] += 1
    
    for region in regions_evolution_pre.keys():
        mtp.plot(np.linspace(-4, 4, num=len(regions_evolution_pre[region] + regions_evolution[region])), 
                 regions_evolution_pre[region] + regions_evolution[region], label=region, linewidth=3 )
    mtp.legend()
    mtp.xlabel('Tempo(s)')
    mtp.ylabel('Número de atributos no top ' + str(top), fontsize=16)
    mtp.axvline(x=0, color = 'black', linewidth = 1, linestyle='dashed')
    mtp.title(rat + ' - Contribuição de cada região na predição (top '+ str(top) + ')', fontsize=16)
    mtp.savefig(rat + '_regions_importance_evolution.png', dpi=200)
    
    mtp.close()  
    to_label = []
    to_label_names = []
    mtp.figure(figsize=(20,14))
    mtp.xlim((-4, 4))
    mtp.ylim((0, len(all_labels) + 1))
    for i in range(0, len(all_labels)):
        x_values = []
        all_ranks = np.array(features_rankings[all_labels[i]])
        all_ranks_pre = np.array(features_rankings_pre[all_labels[i]])
        #print("all ranks", all_ranks)
        mask_below_top = (all_ranks < top)
        mask_above_top = (all_ranks >= top)
        mask_below_top_pre = (all_ranks_pre < top)
        mask_above_top_pre = (all_ranks_pre >= top)
        all_ranks[mask_above_top] = -1
        all_ranks[mask_below_top] = i  
        all_ranks_pre[mask_above_top_pre] = -1
        all_ranks_pre[mask_below_top_pre] = i  
        #if(sum(mask_below_top.astype(int)) > 0 or sum(mask_below_top_pre.astype(int)) > 0):
        #    to_label.append(i)
        #    to_label_names.append(all_labels[i])
        to_label.append(i)
        to_label_names.append(all_labels[i])
        mtp.plot(np.linspace(-4, 4, num=len(np.hstack((all_ranks_pre, all_ranks)))), np.hstack((all_ranks_pre, all_ranks)), 'go', markersize=6)
    mtp.axvline(x=0, color = 'b', linewidth = 1.5, linestyle = 'dashed')
    mtp.yticks(to_label, to_label_names, rotation='horizontal', fontsize=6)
    mtp.xlabel('Tempo (s)')
    mtp.title(rat + ' - Melhores ' + str(top) + ' atributos por janela de tempo', fontsize=16)
    #mtp.legend()    
    mtp.savefig(rat + '_top' + str(top) + '_features_ranking.png', dpi=200)
    
    
    mtp.clf()
    for key in features_rankings.keys():
        ranks_variation = np.array(features_rankings[key])
        mtp.plot(np.arange(0, len(ranks_variation), 1), (ranks_variation < 30).astype(int), 'y*', label=key)
    mtp.close()

sum_better = np.zeros(len(better_curves[0]))
sum_worse = np.zeros(len(worse_curves[0]))
for i in range(0, len(better_curves)):
    sum_better = sum_better + better_curves[i]
    sum_worse = sum_worse + worse_curves[i]
sum_better = sum_better/len(rats)   
sum_worse = sum_worse/len(rats)   
better_std = np.std(np.asarray(better_curves), axis=0)
print(np.shape(np.array(better_std)))
print("std", better_std)
print(np.shape(sum_better))
worse_std = np.std(np.asarray(worse_curves), axis=0)
mtp.figure(figsize=(20,14))
mtp.plot(np.linspace(-4, 4, num=len(sum_better)), sum_better, label='Curva com seleção de atributos')
mtp.plot(np.linspace(-4, 4, num=len(sum_better)), sum_worse, label='Curva sem seleção de atributos')
mtp.legend()
mtp.xlabel('Tempo(s)')
mtp.ylabel('Taxa de acerto [0,1]', fontsize=16)
mtp.axvline(x=0, color = 'black', linewidth = 1, linestyle='dashed')
mtp.title('Taxa de acerto média para a população de ratos', fontsize=16)
mtp.savefig('desempenho_médio_ratos.png', dpi=200)
mtp.close()
mtp.figure(figsize=(20,14))
mtp.ylim((0.4, 1))
mtp.errorbar(np.linspace(-4, 4, num=len(sum_better)), sum_better, yerr=better_std , fmt='--o')
mtp.xlabel('Tempo(s)')
mtp.ylabel('Taxa de acerto [0,1]', fontsize=16)
mtp.axvline(x=0, color = 'black', linewidth = 1, linestyle='dashed')
mtp.title('Taxa de acerto média para a população de ratos com seleção de atributos', fontsize=16)
mtp.savefig('desempenho_médio_ratos_better_deviations.png', dpi=200)
mtp.close()
mtp.figure(figsize=(20,14))
mtp.ylim((0.4, 1))
mtp.errorbar(np.linspace(-4, 4, num=len(sum_better)), sum_worse, yerr=worse_std , fmt='--o')
mtp.xlabel('Tempo(s)')
mtp.ylabel('Taxa de acerto [0,1]', fontsize=16)
mtp.axvline(x=0, color = 'black', linewidth = 1, linestyle='dashed')
mtp.title('Taxa de acerto média para a população de ratos sem seleção de atributos', fontsize=16)
mtp.savefig('desempenho_médio_ratos_worse_deviations.png', dpi=200)
mtp.close()

In [19]:
file = sci2.loadmat('tipo_de_resposta_neuronios/rat1-verde-WD-20150319-60min-255trials-68PC-SPK_responseType')
file['responseType'][0,0]

array([[1, 1, 1, 1, 1, 0]], dtype=uint8)